In [ ]:
!pip install -q accelerate peft transformers torch trl datasets isbnlib tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
import gc
os.environ["TORCHDYNAMO_DISABLE"] = "1"
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    pipeline,
    logging,
)

model_name = "hitmanonholiday/gpt2-4"

#model_name = "gpt2-medium"  # This is the base GPT-2 model with 124M parameters
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token  # Use the end-of-sequence token as padding token
model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')
tokenizer.add_special_tokens({'eos_token': '</s>'})

tokenizer_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

1

In [ ]:
prompt = '''What is the isbn13 number of the book with these details :
    Title: Santuario
    Long Title: Santuario
    Author(s): ['Faulkner, William']
    Language: es
    Published: 1982
    ISBN: 847530088X'''
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[INST] What is the isbn13 number of the book with these details :
    Title: Santuario
    Long Title: Santuario
    Author(s): ['Faulkner, William']
    Language: es
    Published: 1982
    ISBN: 847530088X [/INST]
    The ISBN13 number is 9788475300882 </s>
  


# get 100 seeded random samples

In [ ]:
import random
class BookDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer, max_length=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __getitem__(self, idx):
        # Format the text in a more structured way
        text = self.texts[idx]

        item = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        labels = item['input_ids'].clone()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': item['input_ids'].squeeze(),
            'attention_mask': item['attention_mask'].squeeze(),
            'labels': labels.squeeze()
        }

    def __len__(self):
        return len(self.texts)

    def get_random_samples(self, seed, num_samples):
        random.seed(seed)
        sampled_texts = random.sample(self.texts, num_samples)
        return sampled_texts

In [ ]:
# 3. Prepare the dataset
dataset = load_dataset("P1ayer-1/isbndb-full-database")
dataset = dataset.shuffle(seed=42)
dataset = dataset['train'].select(range(20000))
df = dataset.to_pandas()

# 4. Data preprocessing
# Clean date_published
df['date_published'] = df['date_published'].astype(str).str.extract(r"((?:19|20)\d{2})", expand=False)

# Standardize ISBN13
from isbnlib import to_isbn13
def standardize_isbn(isbn):
    try:
        return to_isbn13(isbn)
    except:
        return None
df['isbn13'] = df['isbn13'].apply(standardize_isbn)


# Remove missing values and duplicates
df = df.dropna(subset=['title', 'authors', 'date_published'])
df = df.drop_duplicates(subset=['title', 'isbn13'])

# 5. Prepare text for tokenization
def prepare_text(row):
    return f'''<s>[INST] <<SYS>> System prompt: This is a isbn13 question-answering session. <</SYS>>
    What is the isbn13 number of the book with these details :
    Title: {row['title']}
    Long Title: {row['title_long']}
    Author(s): {row['authors']}
    Language: {row['language']}
    Published: {row['date_published']}
    ISBN: {row['isbn']} [/INST]
    The ISBN13 number is {row['isbn13']} </s>'''

# Update texts with new format
texts = df.apply(prepare_text, axis=1)




# Create dataset
dataset = BookDataset(texts.tolist(), tokenizer, max_length=256)

# Test the dataset
sample = dataset[0]
print("Input shape:", sample['input_ids'].shape)
print("Attention mask shape:", sample['attention_mask'].shape)
print("Labels shape:", sample['labels'].shape)
print("\nDecoded text:")
print(tokenizer.decode(sample['input_ids'][sample['attention_mask'] == 1]))

# Optional: Check how many tokens are actually being used
num_tokens = (sample['attention_mask'] == 1).sum()
print(f"\nNumber of actual tokens (non-padding): {num_tokens}")

# Set random seeds for reproducibility
import numpy as np
import torch
np.random.seed(42)
torch.manual_seed(42)
# Calculate subset size and validation split
SUBSET_FRACTION = 1  # 1% of full dataset
VALID_FRACTION = 0.1   # 10% of subset for validation
# Convert texts to list if it's a pandas Series
texts_list = texts.tolist() if hasattr(texts, 'tolist') else texts
subset_size = int(len(texts_list) * SUBSET_FRACTION)

# Create subset indices and get subset
subset_indices = np.random.choice(len(texts_list), size=subset_size, replace=False)
texts_subset = [texts_list[i] for i in subset_indices]

# Split subset into train and validation
valid_size = int(len(texts_subset) * VALID_FRACTION)
train_size = len(texts_subset) - valid_size

train_texts = texts_subset[:-valid_size]
valid_texts = texts_subset[-valid_size:]

# Create train and validation datasets
train_dataset = BookDataset(train_texts, tokenizer, max_length=128)
valid_dataset = BookDataset(valid_texts, tokenizer, max_length=128)

# Print dataset sizes
print(f"Full dataset size: {len(texts_list):,}")
print(f"Subset size: {len(texts_subset):,}")
print(f"Training set size: {len(train_dataset):,}")
print(f"Validation set size: {len(valid_dataset):,}")

# Verify samples from both splits
def inspect_dataset(dataset, name):
    sample = dataset[0]
    print(f"\n{name} Sample:")
    print("Input shape:", sample['input_ids'].shape)
    print("Decoded text:")
    print(tokenizer.decode(sample['input_ids'][sample['attention_mask'] == 1]))
    num_tokens = (sample['attention_mask'] == 1).sum()
    print(f"Number of tokens: {num_tokens}")

inspect_dataset(train_dataset, "Training")
inspect_dataset(valid_dataset, "Validation")

README.md:   0%|          | 0.00/671 [00:00<?, ?B/s]

(…)-00000-of-00011-8c76aaae89c32750.parquet:   0%|          | 0.00/230M [00:00<?, ?B/s]

(…)-00001-of-00011-0570d9f1f07ee5ea.parquet:   0%|          | 0.00/222M [00:00<?, ?B/s]

(…)-00002-of-00011-43bde59cf032f28e.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00003-of-00011-cc6f9f6e8311bb1d.parquet:   0%|          | 0.00/300M [00:00<?, ?B/s]

...

(…)-00008-of-00011-bff7d5ac18f4238c.parquet:   0%|          | 0.00/297M [00:00<?, ?B/s]

(…)-00009-of-00011-84ef5d0ca46f90be.parquet:   0%|          | 0.00/230M [00:00<?, ?B/s]

(…)-00010-of-00011-dfef6d372d63bc4c.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28086774 [00:00<?, ? examples/s]

Input shape: torch.Size([256])
Attention mask shape: torch.Size([256])
Labels shape: torch.Size([256])

Decoded text:
<s>[INST] <<SYS>> System prompt: This is a isbn13 question-answering session. <</SYS>>
    What is the isbn13 number of the book with these details :
    Title: Digital Photography All-in-One Desk Reference For Dummies
    Long Title: Digital Photography All-in-One Desk Reference For Dummies
    Author(s): ['Busch, David D.']
    Language: en
    Published: 2008
    ISBN: 0470401958 [/INST]
    The ISBN13 number is 9780470401958 </s>

Number of actual tokens (non-padding): 146
Full dataset size: 18,745
Subset size: 18,745
Training set size: 16,871
Validation set size: 1,874

Training Sample:
Input shape: torch.Size([128])
Decoded text:
<s>[INST] <<SYS>> System prompt: This is a isbn13 question-answering session. <</SYS>>
    What is the isbn13 number of the book with these details :
    Title: Santuario
    Long Title: Santuario
    Author(s): ['Faulkner, William']
    

In [ ]:
dataset_sample = train_dataset.get_random_samples(50,200)

In [ ]:
text = '''
<s>[INST] <<SYS>> System prompt: This is a isbn13 question-answering session. <</SYS>>
    What is the isbn13 number of the book with these details :
    Title: Santuario
    Long Title: Santuario
    Author(s): ['Faulkner, William']
    Language: es
    Published: 1982
    ISBN: 847530088X [/INST]
    The ISBN13 number is 9788475300887 </s>
'''

In [ ]:
import re
pattern = r"<</SYS>>(.*?)\[/INST]"
match = re.search(pattern, text, re.DOTALL)

# Extract the matched string if found
result = match.group(1).strip() if match else None

print(result)

What is the isbn13 number of the book with these details :
    Title: Santuario
    Long Title: Santuario
    Author(s): ['Faulkner, William']
    Language: es
    Published: 1982
    ISBN: 847530088X


In [ ]:
type(dataset_sample[15])

str

In [ ]:
dataset_sample[15]

"<s>[INST] <<SYS>> System prompt: This is a isbn13 question-answering session. <</SYS>>\n    What is the isbn13 number of the book with these details :\n    Title: My Emily Dickinson (New Directions Paperbook)\n    Long Title: My Emily Dickinson (New Directions Paperbook)\n    Author(s): ['Howe, Susan']\n    Language: en\n    Published: 2007\n    ISBN: 0811223345 [/INST]\n    The ISBN13 number is 9780811223348 </s>"

In [ ]:
def get_isbn13(text):
    isbn13_match = re.search(r"\b\d{13}\b", text)
    return isbn13_match.group(0) if isbn13_match else None
def get_question(text):
    pattern = r"<</SYS>>(.*?)\[/INST]"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else None

device = 0 if torch.cuda.is_available() else -1

def infer(prompt):
  isbn13_from_prompt = get_isbn13(prompt)
  q = get_question(prompt)
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, truncation=True, device=device)
  result = pipe(f"[INST] {q} [/INST]")
  result = result[0]['generated_text'].split("</s>")[0] + "</s>"
  isbn13_from_response = get_isbn13(result)
  return isbn13_from_prompt == isbn13_from_response

In [ ]:
def get_model_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name,max_length=200, truncation=True)
    model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')
    tokenizer.add_special_tokens({'eos_token': '</s>'})
    return model, tokenizer

In [ ]:
from tqdm import tqdm
import gc
model_names = [
    "hitmanonholiday/gpt2-4",
    "hitmanonholiday/gpt2-8",
    "hitmanonholiday/gpt2-16",
    "hitmanonholiday/gpt2-medium-4",
    "hitmanonholiday/gpt2-medium-8",
    "hitmanonholiday/gpt2-medium-16",
    "hitmanonholiday/gpt2-large-4",
    "hitmanonholiday/gpt2-large-8",
    "hitmanonholiday/gpt2-large-16",
]

hallucination_acc = {}
for model_name in model_names:
    model, tokenizer = get_model_tokenizer(model_name)
    count = 0
    for x in tqdm(dataset_sample):
      if infer(x):
        count = count + 1
    print(model_name,"hallucination accuracy :",count/len(dataset_sample))
    hallucination_acc[model_name] = count/len(dataset_sample)
    del model
    gc.collect()
    torch.cuda.empty_cache()
    print("Cuda memory flushed\n")

100%|██████████| 200/200 [05:44<00:00,  1.72s/it]


hitmanonholiday/gpt2-4 hallucination accuracy : 0.05
Cuda memory flushed



tokenizer_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/939 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/47.2M [00:00<?, ?B/s]

100%|██████████| 200/200 [16:01<00:00,  4.81s/it]


hitmanonholiday/gpt2-large-16 hallucination accuracy : 0.075
Cuda memory flushed



In [ ]:
hallucination_acc

{'hitmanonholiday/gpt2-4': 0.05,
 'hitmanonholiday/gpt2-8': 0.06,
 'hitmanonholiday/gpt2-16': 0.07,
 'hitmanonholiday/gpt2-medium-4': 0.05,
 'hitmanonholiday/gpt2-medium-8': 0.05,
 'hitmanonholiday/gpt2-medium-16': 0.06,
 'hitmanonholiday/gpt2-large-4': 0.035,
 'hitmanonholiday/gpt2-large-8': 0.065,
 'hitmanonholiday/gpt2-large-16': 0.075}